# Python scikit-learn Machine Learning Workflow 

'''
Python scikit-learn Machine Learning Workflow

In this 1.5 h session you will learn how to build an efficient, reusable workflow that starts with a pandas DataFrame and ends with a trained scikit-learn linear regression model.

Pandas
-	Overview of how to use pandas to read, describe, clean and prepare a dataset for machine learning
scikit-learn
-	Handle missing data
-	Create training and testing datasets
-	Impute missing values
-	Perform linear regression for a dependent variable of type float (target) and independent variables of type float or integer (features)
-	Create a workflow pipeline
-	Perform n-fold cross validation on the entire pipeline
-	Tune the hyperparameters of the model using grid search
-	Perform feature selection
'''

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Set global parameters
filename = 'lunch_and_learn.csv'

In [3]:
# Read data file into a pandas DataFrame
data = pd.read_csv(filename)

In [4]:
data.shape

(5000, 14)

In [5]:
data.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'Y'],
      dtype='object')

In [6]:
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,Y
0,-0.562021,1.043343,0.696816,-0.189681,0.633079,0.342155,-0.365107,2.013158,-4.627122,0.291391,-1.359770,9.299416,0.555481,60.993496
1,-0.020557,1.305215,0.611167,-0.047938,-0.032016,0.021054,-0.572075,-2.308974,2.390728,0.159002,-0.452501,3.256993,-0.004232,60.109317
2,2.769638,0.340073,-0.434665,-2.246398,-0.637302,-0.480945,0.226892,-3.395083,-0.408775,0.356323,-0.066215,-2.938287,-0.463020,91.812152
3,-6.988279,0.523329,-0.843785,1.216130,0.001059,-0.552828,2.392279,3.063360,-2.461017,1.036167,-0.406774,1.274303,0.535255,-25.207493
4,-0.042973,-0.441435,-0.164827,0.929786,-0.057137,-0.326923,-0.637193,0.642685,3.110585,6.674638,-0.463073,-1.857846,-1.573654,68.891226
